# 第十章 Go 的並行 (Go Concurrency)

## 章節概述

並行程式設計是 Go 語言的核心特色之一。Go 透過 goroutine 和 channel 提供了優雅且強大的並行程式設計模型，讓開發者能夠輕鬆地編寫高效能的並行程式。

### 本章重點內容
- **何時該使用並行**: 判斷並行的適用場景
- **goroutine**: Go 的輕量級執行緒
- **channel**: goroutine 間的通訊機制
- **select**: 多路通訊控制
- **並行模式**: 常見的並行設計模式
- **goroutine 管理**: 生命週期和清理
- **同步原語**: WaitGroup, Once, Mutex
- **效能考量**: 何時使用 channel vs mutex

### Go 並行模型的核心理念
> "Don't communicate by sharing memory; share memory by communicating."
> 
> 不要透過共享記憶體來通訊；而要透過通訊來共享記憶體。

這句話概括了 Go 並行程式設計的核心哲學：優先使用 channel 進行通訊，而不是直接操作共享變數。

In [ ]:
// Go 並行程式設計基礎示範
package main

import (
    "fmt"
    "math/rand"
    "runtime"
    "sync"
    "time"
)

func main() {
    fmt.Println("=== Go 並行程式設計基礎 ===")
    
    // 1. 基本 goroutine 概念
    fmt.Println("\n1. 基本 goroutine 示範:")
    demonstrateBasicGoroutines()
    
    // 2. channel 基礎
    fmt.Println("\n2. Channel 基礎:")
    demonstrateBasicChannels()
    
    // 3. 有緩衝 vs 無緩衝 channel
    fmt.Println("\n3. 緩衝 Channel:")
    demonstrateBufferedChannels()
    
    // 4. select 語句
    fmt.Println("\n4. Select 語句:")
    demonstrateSelect()
    
    fmt.Printf("\n當前系統資訊: CPU核心數=%d, goroutines數量=%d\n", 
        runtime.NumCPU(), runtime.NumGoroutine())
}

// 示範基本 goroutine 使用
func demonstrateBasicGoroutines() {
    fmt.Println("  啟動前 goroutines:", runtime.NumGoroutine())
    
    // 使用 WaitGroup 等待 goroutine 完成
    var wg sync.WaitGroup
    
    // 啟動多個 goroutine
    for i := 1; i <= 3; i++ {
        wg.Add(1) // 增加等待計數
        
        go func(id int) {
            defer wg.Done() // 完成時減少計數
            
            // 模擬一些工作
            sleepTime := time.Duration(rand.Intn(1000)) * time.Millisecond
            fmt.Printf("    Goroutine %d: 開始工作，預計耗時 %v\n", id, sleepTime)
            time.Sleep(sleepTime)
            fmt.Printf("    Goroutine %d: 工作完成\n", id)
        }(i) // 傳遞 i 的值給 goroutine
    }
    
    fmt.Println("  啟動後 goroutines:", runtime.NumGoroutine())
    
    // 等待所有 goroutine 完成
    wg.Wait()
    fmt.Println("  所有工作完成!")
}

// 示範基本 channel 使用
func demonstrateBasicChannels() {
    // 建立一個無緩衝的 channel
    messages := make(chan string)
    
    // 在 goroutine 中發送訊息
    go func() {
        messages <- "Hello from goroutine!"
        fmt.Println("    訊息已發送")
    }()
    
    // 在主 goroutine 中接收訊息
    fmt.Println("    等待接收訊息...")
    msg := <-messages
    fmt.Printf("    收到訊息: %s\n", msg)
    
    // 示範 channel 的關閉和檢查
    numbers := make(chan int)
    
    go func() {
        defer close(numbers) // 關閉 channel
        
        for i := 1; i <= 5; i++ {
            numbers <- i
            fmt.Printf("    發送數字: %d\n", i)
            time.Sleep(100 * time.Millisecond)
        }
        fmt.Println("    數字發送完畢，關閉 channel")
    }()
    
    // 使用 for range 接收直到 channel 關閉
    fmt.Println("    開始接收數字:")
    for num := range numbers {
        fmt.Printf("    接收到數字: %d\n", num)
    }
    fmt.Println("    Channel 已關閉，接收完畢")
}

// 示範有緩衝的 channel
func demonstrateBufferedChannels() {
    // 建立有緩衝的 channel（容量為 3）
    bufferedCh := make(chan string, 3)
    
    fmt.Println("    緩衝 channel 容量:", cap(bufferedCh))
    
    // 發送訊息（不會阻塞，因為有緩衝）
    bufferedCh <- "訊息1"
    fmt.Printf("    發送後長度: %d/%d\n", len(bufferedCh), cap(bufferedCh))
    
    bufferedCh <- "訊息2"
    bufferedCh <- "訊息3"
    fmt.Printf("    發送3個訊息後長度: %d/%d\n", len(bufferedCh), cap(bufferedCh))
    
    // 接收訊息
    for i := 0; i < 3; i++ {
        msg := <-bufferedCh
        fmt.Printf("    接收: %s, 剩餘長度: %d\n", msg, len(bufferedCh))
    }
    
    // 示範緩衝 channel 的非阻塞特性
    go func() {
        for i := 1; i <= 5; i++ {
            bufferedCh <- fmt.Sprintf("批次訊息%d", i)
            fmt.Printf("    快速發送: 批次訊息%d\n", i)
        }
        close(bufferedCh)
    }()
    
    // 延遲接收
    time.Sleep(500 * time.Millisecond)
    fmt.Println("    開始批次接收:")
    for msg := range bufferedCh {
        fmt.Printf("    批次接收: %s\n", msg)
        time.Sleep(200 * time.Millisecond) // 慢速接收
    }
}

// 示範 select 語句的使用
func demonstrateSelect() {
    ch1 := make(chan string)
    ch2 := make(chan string)
    done := make(chan bool)
    
    // 第一個 goroutine
    go func() {
        time.Sleep(1 * time.Second)
        ch1 <- "來自 channel 1 的訊息"
    }()
    
    // 第二個 goroutine
    go func() {
        time.Sleep(2 * time.Second)
        ch2 <- "來自 channel 2 的訊息"
    }()
    
    // 第三個 goroutine（超時控制）
    go func() {
        time.Sleep(3 * time.Second)
        done <- true
    }()
    
    receivedCount := 0
    timeout := time.After(2500 * time.Millisecond) // 2.5秒超時
    
    fmt.Println("    使用 select 等待多個 channel...")
    
    for receivedCount < 2 {
        select {
        case msg1 := <-ch1:
            fmt.Printf("    從 ch1 接收: %s\n", msg1)
            receivedCount++
            
        case msg2 := <-ch2:
            fmt.Printf("    從 ch2 接收: %s\n", msg2)
            receivedCount++
            
        case <-timeout:
            fmt.Println("    接收超時！")
            return
            
        case <-done:
            fmt.Println("    收到完成信號")
            return
            
        default:
            // 非阻塞操作
            fmt.Println("    目前沒有訊息可接收...")
            time.Sleep(500 * time.Millisecond)
        }
    }
    
    fmt.Printf("    總共接收了 %d 個訊息\n", receivedCount)
}